This file further process output from VEP and split variants into mapped and unmapped to uniprot.

Specifically,

With the output from VEP, do:
1. filter the output file and retain the rows where the SWISSPROT and TREMBL columns are not be both empty. 
2. There should be only one identifier in SWISSPROT and might be multiple identifiers in TREMBL (if so separated by ","). 
3. If the SWISSPROT or TREMBL identifiers are in the format of XXXXXX.XXX(for example P43489.17), only retain the content before "." (P43489). 
4. Then choose the best UniProt among SWISSPROT and TREMBL identifiers for each row, prioritizing SWISSPROT over TREMBL, and choosing the longest TREMBL identifier if there is no SWISSPROT. 
5. Naming the column containing the best UniProt as UNIPROT. Rename the Protein_position column as UNIPROT_position.

(take coding_indels as an example:)

Parsing the Amino_acids column in the VEP output file into two columns: REF_AA and ALT_AA. For example, if the value in Amino_acids is "RLTQTV/X",
the values in REF_AA and ALT_AA should be "RLTQTV" and "X" respectively.

Retain the six columns in VEP output: #Uploaded_variation, UNIPROT, UNIPROT_position, REF_AA, ALT_AA and Consequence, and map the VEP output to
the coding indels using #Uploaded_variation as key. 

Finally you should get a text file named mapped_coding_indel.txt with 18 columns: 'CHROM', 'POS', 'ID'(HGMD_ID), 'REF'(REF_N), 'ALT'(ALT_N),'REF_AA', 'ALT_AA','UNIPROT', 'UNIPROT_position','CLASS'(Variant_class),'GENE','STRAND', 'DB'(dbSNP), 'PHEN'(Phenotype),'RANKSCORE','DNA'(RefSeq_DNA_record), 'PROT'(RefSeq_protein_record), 'Consequence'

Also you should get unmapped_coding_indel.txt with 13 columns (missing REF_AA, ALT_AA,
UNIPROT, UNIPROT_position, and Consequence).

In [ ]:
import pandas as pd
import numpy as np
coding_indels = pd.read_csv("/share/yu/mw2243/HGMD/coding_indels.tsv",sep="\t")
VEP_res = pd.read_csv("/share/yu/mw2243/HGMD/indels_VEP_results.txt",sep='\t')
#filter out both '-' in 'SWISSPROT','TREMBL'
filter = VEP_res.loc[~((VEP_res['SWISSPROT'].values=='-')&(VEP_res['TREMBL'].values=='-'))]
# Step 1: Remove version numbers (everything after '.')
# Step 2: Select best UniProt ID with priority rules
filter['SWISSPROT'] = filter['SWISSPROT'].str.split('.').str[0]
filter['TREMBL'] = filter['TREMBL'].str.split('.').str[0]
filter['UNIPROT'] = np.where(
    filter['SWISSPROT']!='-',  # Priority 1: SWISSPROT exists
    filter['SWISSPROT'],
    filter['TREMBL']
)
# Step 3: Rename Protein_position column
if 'Protein_position' in filter.columns:
    filter = filter.rename(columns={'Protein_position': 'UNIPROT_position'})


filter[['REF_AA', 'ALT_AA']] = filter['Amino_acids'].str.split('/', expand=True)
filter = filter[['UNIPROT','UNIPROT_position','REF_AA', 'ALT_AA','Consequence']]
df_joined = pd.concat([filter.reset_index(drop=True), coding_indels.iloc[filter.index.values,:].reset_index(drop=True)],axis=1)
df_joined = df_joined[['CHROM', 'POS', 'ID', 'REF', 'ALT','REF_AA', 'ALT_AA','UNIPROT', 'UNIPROT_position','CLASS','GENE','STRAND', 'DB', 'PHEN','RANKSCORE','DNA', 'PROT', 'Consequence']]
df_joined.to_csv('/share/yu/mw2243/HGMD/processed/mapped_coding_indel.txt')

mask = ~coding_indels.reset_index(drop=True).index.isin(filter.index)
filtered_indels = coding_indels.reset_index(drop=True)[mask].reset_index(drop=True)
filtered_indels.to_csv("/share/yu/mw2243/HGMD/processed/unmapped_coding_indel.txt")


In [ ]:
#missense
import pandas as pd
import numpy as np
VEP_res_path = "/share/yu/mw2243/HGMD/missense_VEP_results.txt"
missense = pd.read_csv("/share/yu/mw2243/HGMD/missense_variants.tsv",sep="\t")
VEP_res = pd.read_csv(VEP_res_path,sep='\t')
#filter out both '-' in 'SWISSPROT','TREMBL'
filter = VEP_res.loc[~((VEP_res['SWISSPROT'].values=='-')&(VEP_res['TREMBL'].values=='-'))]
filter['SWISSPROT'] = filter['SWISSPROT'].str.split('.').str[0]
filter['TREMBL'] = filter['TREMBL'].str.split('.').str[0]
filter['UNIPROT'] = np.where(
    filter['SWISSPROT']!='-',  # Priority 1: SWISSPROT exists
    filter['SWISSPROT'],
    filter['TREMBL']
)
if 'Protein_position' in filter.columns:
    filter = filter.rename(columns={'Protein_position': 'UNIPROT_position'})

filter[['REF_AA', 'ALT_AA']] = filter['Amino_acids'].str.split('/', expand=True)
filter = filter[['UNIPROT','UNIPROT_position','REF_AA', 'ALT_AA','Consequence']]
df_joined = pd.concat([filter.reset_index(drop=True), missense.iloc[filter.index.values,:].reset_index(drop=True)],axis=1)
df_joined = df_joined[['CHROM', 'POS', 'ID', 'REF', 'ALT','REF_AA', 'ALT_AA','UNIPROT', 'UNIPROT_position','CLASS','GENE','STRAND', 'DB', 'PHEN','RANKSCORE','DNA', 'PROT', 'Consequence']]
df_joined.to_csv('/share/yu/mw2243/HGMD/processed/mapped_missense.txt')

mask = ~missense.reset_index(drop=True).index.isin(filter.index)
filtered_indels = missense.reset_index(drop=True)[mask].reset_index(drop=True)
filtered_indels.to_csv("/share/yu/mw2243/HGMD/processed/unmapped_missense.txt")


In [ ]:
#nonsense
import pandas as pd
import numpy as np
VEP_res_path = "/share/yu/mw2243/HGMD/nonsense_VEP_results.txt"
nonsense = pd.read_csv("/share/yu/mw2243/HGMD/nonsense_variants.tsv",sep="\t")
VEP_res = pd.read_csv(VEP_res_path,sep='\t')
#filter out both '-' in 'SWISSPROT','TREMBL'
filter = VEP_res.loc[~((VEP_res['SWISSPROT'].values=='-')&(VEP_res['TREMBL'].values=='-'))]
filter['SWISSPROT'] = filter['SWISSPROT'].str.split('.').str[0]
filter['TREMBL'] = filter['TREMBL'].str.split('.').str[0]
filter['UNIPROT'] = np.where(
    filter['SWISSPROT']!='-',  # Priority 1: SWISSPROT exists
    filter['SWISSPROT'],
    filter['TREMBL']
)
if 'Protein_position' in filter.columns:
    filter = filter.rename(columns={'Protein_position': 'UNIPROT_position'})


filter[['REF_AA', 'ALT_AA']] = filter['Amino_acids'].str.split('/', expand=True)
filter = filter[['UNIPROT','UNIPROT_position','REF_AA', 'ALT_AA','Consequence']]
df_joined = pd.concat([filter.reset_index(drop=True), nonsense.iloc[filter.index.values,:].reset_index(drop=True)],axis=1)
df_joined = df_joined[['CHROM', 'POS', 'ID', 'REF', 'ALT','REF_AA', 'ALT_AA','UNIPROT', 'UNIPROT_position','CLASS','GENE','STRAND', 'DB', 'PHEN','RANKSCORE','DNA', 'PROT', 'Consequence']]
df_joined.to_csv('/share/yu/mw2243/HGMD/processed/mapped_nonsense.txt')

mask = ~nonsense.reset_index(drop=True).index.isin(filter.index)
filtered_indels = nonsense.reset_index(drop=True)[mask].reset_index(drop=True)
filtered_indels.to_csv("/share/yu/mw2243/HGMD/processed/unmapped_nonsense.txt")
